# Sentiment Analysis

Congrats, you finished the part on the data preparation, and we can now move on to the more exciting part of using RNNs/LSTMs to process sequential data! But be careful, even if the previous notebook might seem a little bit boring, it is of great importance. We switched from images to text data in this exercise, and remember the first steps that we did in our class were also data related, and they were essential for all the following exercises. So naturally, since we switched to text data in this exercise, make sure you have a good understanding of how the data has been prepared.

For the last I2DL exercise, we want to make use of Recurrent Neural Networks (RNNs) to process sequential data. We will stick with the same dataset we have been looking at in the previous notebook, namely the [IMDb](https://ai.stanford.edu/~amaas/data/sentiment/) sentiment analysis dataset that contains positive and negative movie reviews.

<p class="aligncenter">
    <img src="images/IMDB.jpg" alt="centered image" />
</p>

Sentiment analysis is the task of predicting the sentiment of a text. In this notebook, you will train a network to process reviews from the dataset and evaluate whether it has been a positive or a negative review. Below are two examples:

![Screenshot%20from%202021-01-25%2008-29-35.png](attachment:Screenshot%20from%202021-01-25%2008-29-35.png)


## (Optional) Mount folder in Colab

Uncomment the following cell to mount your gdrive if you are using the notebook in google colab:

In [1]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "i2dl" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g exercise_11) is given.

# """
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/i2dl/exercise_11'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))
# """

Mounted at /content/gdrive
['1_text_preprocessing_and_embedding.ipynb', '2_sentiment_analysis.ipynb', 'Optional-recurrent_neural_networks.ipynb', 'exercise_code', 'images', 'models']


### Set up PyTorch environment in colab
- (OPTIONAL) Enable GPU via Runtime --> Change runtime type --> GPU
- Uncomment the following cell if you are using the notebook in google colab:

In [2]:
# Optional: install correct libraries in google colab
!python -m pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install tensorboard==2.8.0
!python -m pip install pytorch-lightning==1.6.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 772.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.11.0+cu113 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.11.0+cu113 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# 0. Setup

As always, we first import some packages to setup the notebook.

In [3]:
import os
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_

from exercise_code.rnn.sentiment_dataset import (
    download_data,
    load_sentiment_data,
    load_vocab,
    SentimentDataset,
    collate
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# 1. Loading Data

As we have learned from the notebook 1, this time we not only load the raw data, but also have the corresponding vocabulary. Let us load the data that we prepared for you:

In [4]:
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(i2dl_exercises_path, "datasets", "SentimentData")
base_dir = download_data(data_root)
vocab = load_vocab(base_dir)
train_data, val_data, test_data = load_sentiment_data(base_dir, vocab)

print("number of training samples: {}".format(len(train_data)))
print("number of validation samples: {}".format(len(val_data)))
print("number of test samples: {}".format(len(test_data)))

number of training samples: 9154
number of validation samples: 3133
number of test samples: 3083


## Dataset Samples

Our raw data consists of tuples `(raw_text, token_list, token_indices, label)`. Let's sample some relatively short texts from our dataset to have a sense how it looks like:

In [5]:
sample_data0 = [datum for datum in train_data if len(datum[1]) < 20 and datum[-1] == 0] # negative
sample_data1 = [datum for datum in train_data if len(datum[1]) < 20 and datum[-1] == 1] # positive

# we sample 2 tuples each from positive set and negative set
sample_data = random.sample(sample_data0, 4) + random.sample(sample_data1, 2)
for text, tokens, indices, label in sample_data:
    print('Text: \n {}\n'.format(text))
    print('Tokens: \n {}\n'.format(tokens))
    print('Indices: \n {}\n'.format(indices))
    print('Label:\n {}\n'.format(label))
    print()

Text: 
 Ming The Merciless does a little Bardwork and a movie most foul!

Tokens: 
 ['ming', 'the', 'merciless', 'does', 'a', 'little', 'bardwork', 'and', 'a', 'movie', 'most', 'foul']

Indices: 
 [1, 2, 1, 142, 3, 121, 1, 4, 3, 13, 99, 2850]

Label:
 0


Text: 
 Long, boring, blasphemous. Never have I been so glad to see ending credits roll.

Tokens: 
 ['long', 'boring', 'blasphemous', 'never', 'have', 'i', 'been', 'so', 'glad', 'to', 'see', 'ending', 'credits', 'roll']

Indices: 
 [209, 238, 1, 115, 26, 7, 90, 34, 801, 6, 55, 256, 850, 1403]

Label:
 0


Text: 
 A rating of "1" does not begin to express how dull, depressing and relentlessly bad this movie is.

Tokens: 
 ['a', 'rating', 'of', '1', 'does', 'not', 'begin', 'to', 'express', 'how', 'dull', 'depressing', 'and', 'relentlessly', 'bad', 'this', 'movie', 'is']

Indices: 
 [3, 512, 5, 241, 142, 24, 1095, 6, 2747, 83, 552, 2227, 4, 1, 59, 10, 13, 9]

Label:
 0


Text: 
 Comment this movie is impossible. Is terrible, very improba

## Checking the Vocabulary

In the previous notebook, we discussed the need of a vocabulary for mapping words to unique integer IDs. Instead of creating the vocabulary manually, we provide you with the vocabulary. Let's have a look at some samples from the vocabulary of the dataset:

In [6]:
print('Vocabulary size:', len(vocab), '\n\n  Sample words\n{}'.format('-' * 20))
sample_words = random.sample(list(vocab.keys()), 10)
for word in sample_words:
    print(' {}'.format(word))

Vocabulary size: 5002 

  Sample words
--------------------
 uplifting
 mix
 forward
 recognized
 text
 italy
 concerning
 streisand
 suggested
 being


Also we saw that there are already indices in the raw data that we loaded. We can check if the indices in the vocabulary match the raw data using the last sentence in `sample_data`. Words that are not in the vocabulary are assigned to the symbol `<unk>`. The output of the following cell should be the same as the indices in the last example of our loaded raw data:

In [7]:
# Last sample from above
(text, tokens, indices, label) = sample_data[-1]
print('Text: \n {}\n'.format(text))
print('Tokens: \n {}\n'.format(tokens))
print('Indices: \n {}\n'.format(indices))
print('Label:\n {}\n'.format(label))

# Compare with the vocabulary
print('Indices drawn from vocabulary: \n {}\n'.format([vocab.get(x, vocab['<unk>']) for x in sample_data[-1][1]]))

Text: 
 I don't know why I like this movie so well, but I never get tired of watching it.

Tokens: 
 ['i', 'don', 't', 'know', 'why', 'i', 'like', 'this', 'movie', 'so', 'well', 'but', 'i', 'never', 'get', 'tired', 'of', 'watching', 'it']

Indices: 
 [7, 74, 23, 126, 138, 7, 32, 10, 13, 34, 68, 17, 7, 115, 82, 1225, 5, 116, 8]

Label:
 1

Indices drawn from vocabulary: 
 [7, 74, 23, 126, 138, 7, 32, 10, 13, 34, 68, 17, 7, 115, 82, 1225, 5, 116, 8]



## Wrapping to PyTorch Datasets

Great, the raw data is loaded properly and the vocabulary is matching. Let us wrap our data in a PyTorch dataset. For more details, check out the previous notebook and the corresponding dataset class defined in `exercise_code/rnn/sentiment_dataset.py`.

In [8]:
# Define a Dataset Class for train, val and test set
train_dataset = SentimentDataset(train_data)
val_dataset = SentimentDataset(val_data)
test_dataset = SentimentDataset(test_data)

# 2. Creating a Sentiment Classifier

After we have loaded the data, it is time to define a model and start training and testing.

## Evaluation Metrics

Since we just need to predict positive or negative, we can use `binary cross-entropy loss` to train our model. And accuracy can be used to assess our model's performance. We will use the following evaluation model to compute the accuracy.

In [9]:
bce_loss = nn.BCELoss()
test_loss_history = []
@torch.no_grad()
def compute_accuracy(model, data_loader, test_loss_history):
    corrects = 0
    total = 0
    device = next(model.parameters()).device

    bce_loss = nn.BCELoss()


    for i, x in enumerate(data_loader, test_loss_history):
        input = x['data'].to(device)
        lengths = x['lengths']
        label = x['label'].to(device)
        pred = model(input, lengths)

        loss = bce_loss(pred, label)
        test_loss_history += loss.item()

        corrects += ((pred > 0.5) == label).sum().item()

        total += label.numel()

        if i > 0  and i % 100 == 0:
            print('Step {} / {}'.format(i, len(data_loader)))

    return corrects / total, test_loss_history

## Step 1: Design your own model

In this part, you need to create a classifier using the Embedding layers you implemented in the first notebook and LSTM. For the LSTM, you may also use the PyTorch implementation.

![Screenshot%20from%202021-01-25%2008-43-02.png](attachment:Screenshot%20from%202021-01-25%2008-43-02.png)





<div class="alert alert-info">
    <h3>Task: Implement a Classifier</h3>
    
   Go to <code>exercise_code/rnn/text_classifiers.py</code> and implement the <code>RNNClassifier</code>. In the skeleton code, we inherited <code>nn.Module</code>. You can also inherit <code>LightningModule</code> if you want to use PyTorch Lightning.
</div>

This file is mostly empty but contains the expected class name, and the methods that your model needs to implement (only `forward()` basically).
The only rules your model design has to follow are:
* Inherit from `torch.nn.Module` or `pytorch_lightning.LightningModule`
* Perform the forward pass in `forward()`.
* Have less than 2 million parameters
* Have a model size of less than 50MB after saving

After you finished, edit the below cell to make sure your implementation is correct. You should define the model yourself, which should be small enough (2 Mio. parameters) and have correct output format.


In [10]:
from exercise_code.rnn.tests import classifier_test, parameter_test
from exercise_code.rnn.text_classifiers import RNNClassifier

########################################################################
# TODO - Create a Model                                               #
########################################################################

model = RNNClassifier(num_embeddings = len(vocab), embedding_dim = 50, hidden_size = 256)
# pl_model = pl_RNNClassifier(num_embeddings = len(vocab), embedding_dim = 16, hidden_size = 512 )


########################################################################
#                           END OF YOUR CODE                           #
########################################################################

# Check whether your model is sufficiently small and have a correct output format
parameter_test(model), classifier_test(model, len(vocab))

Total number of parameters: 565749
Your model is sufficiently small :)
All output tests are passed :)!


(True, True)

## Step 2: Train your own model

In this section, you need to train the classifier you created. Below, you can see some setup code we provided to you. Note the **collate function** used with the `DataLoader`. If you forgot why we need the collate function here, check this out in Notebook 1.

You are free to change the below configs (e.g. batch size, device setting etc.) as you wish.

In [11]:
# Training configs
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using {}...\n'.format(device))

# Move model to the device we are using
model = model.to(device)

# To tackle with the exploding gradient problem, you may want to set gclip and use clip_grad_norm_
# see the optional notebook for the explanation
gclip = None

# data set
train_dataset = SentimentDataset(train_data)
val_dataset = SentimentDataset(val_data)
test_dataset = SentimentDataset(test_data)


# Dataloaders, note the collate function
train_loader = DataLoader(
  train_dataset, batch_size=16, collate_fn=collate, drop_last=True
)
val_loader = DataLoader(
  val_dataset, batch_size=16, collate_fn=collate, drop_last=False
)


Using cuda...



<div class="alert alert-info">
<h3>Task: Implement Training</h3>
    <p>
        In the below cell, you are expected to implement your training loop to train your model. You can use the training loader provided above for iterating over the data. If you want to evaluate your model periodically, you may use the validation loader provided above. You can use pure PyTorch or PyTorch Lightning.
   </p>
</div>

**Hints :**
* Use `torch.nn.BCELoss` as loss function
* Revise the previous exercises if you get stuck.


In [12]:
########################################################################
#                     TODO - Train Your Model                          #
########################################################################

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
import numpy as np

hparams = {
    "epochs" : 50,

    # optimizer
    "learning_rate" : 1e-4,
    "weight_decay": 1e-9,
    'momentum': 0.9,

    # clipping
    'gclip' : 10
}


# moveing the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

# initializing thel ist for storing the loss and accuracy
train_loss_history = []
val_loss_history = []
lr_history = []

# get the number of batches
def get_num_batches(dataloader, data):
  return int(len(batch) // dataloader.batch_size)

# TODO : Learning Scheduler : https://velog.io/@idj7183/Competition%EC%97%90%EC%84%9C-%EC%83%88%EB%A1%9C-%EB%B0%B0%EC%9A%B4-%EA%B2%83
def get_scheduler(optimizer, step_size, gamma):
  return torch.optim.lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)

# create progress bar
def create_tqdm_bar(iterable, desc):
  return tqdm(enumerate(iterable), total=len(iterable), ncols = 150, desc = desc, leave = False)

# Training
def train_model(model, train_loader, val_loader, loss_func, optim, writer, gclip=30, epochs = 50):
  optimizer = optim
  # scheduler = get_scheduler(optimizer = optimizer, step_size = epochs * len(train_loader) / 20, gamma = 0.6)
  # scheduler = get_scheduler(optimizer = optimizer, step_size = 50, gamma = 0.7)
  scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.6, min_lr = 1e-8)
  # scheduler = CosineAnnealingWarmRestarts(optimizer,
                                        # T_0 = 8,# Number of iterations for the first restart
                                        # T_mult = 1, # A factor increases TiTi​ after a restart
                                        # eta_min = 1e-4) # Minimum learning rate

  for epoch in range(epochs):
    # train
    model.train()
    train_running_loss = 0  # 출력용
    for train_iteration, train_batch in enumerate(train_loader):
      #
      seq = train_batch['data'].to(device)
      label = train_batch['label'].to(device)
      seq_lens = train_batch['lengths']

      optimizer.zero_grad()
      # print('seq : ', seq)
      # print('seq len : ', seq_lens)

      pred = model(seq, seq_lens)
      loss = loss_func(pred, label)
      train_running_loss += loss.item()



      loss.backward()

      # clipping the gradient
      torch.nn.utils.clip_grad_norm_(model.parameters(), 20)

      optimizer.step()


      # if ((train_iteration + 1) % 100) == 0:
      # print('-------step {}/{}, train loss : {}'.format(train_iteration + 1, len(train_loader), loss.item()))

    # update step based scheduler
    # scheduler.step()

    # validation
    model.eval()
    validation_running_loss = 0 # 출력용
    with torch.no_grad():
      # for val_iteration, val_batch in validation_loop:
      for val_iteration, val_batch in enumerate(val_loader):
        seq = val_batch['data'].to(device)
        label = val_batch['label'].to(device)
        seq_lens = val_batch['lengths']

        pred = model(seq, seq_lens)
        loss = loss_func(pred, label)

        # validation_loss += loss.item()
        validation_running_loss += loss.item()




        # if ((val_iteration + 1) % 100) == 0:
          # print('-------step {}/{}, validation loss : {}'.format(val_iteration + 1, len(val_loader), loss.item()))

    # save loss for plotting
    train_loss_history.append(train_running_loss / len(train_loader))
    val_loss_history.append(validation_running_loss / len(val_loader))
    lr_history.append(optimizer.param_groups[0]['lr'])


    # learning rate step when plateau <- Plateau Scheduler
    scheduler.step(validation_running_loss / len(val_loader))

    # print current lr
    print('current learning rate : ',  scheduler._last_lr)

    # print current validation loss
    print("EPOCH %04d / %04d | %12s : %.8f | %12s : %0.8f"  % (epoch + 1, epochs, 'VALIDATION LOSS', validation_running_loss / len(val_loader) , 'TRAIN LOSS', train_running_loss / len(train_loader)))

# Tensorboard, summarywriter
from torch.utils.tensorboard import SummaryWriter
base_dir = '/content/gdrive/MyDrive/i2dl/exercise_10'
log_dir = os.path.join(base_dir, 'log')
writer = SummaryWriter(log_dir = log_dir)


# optimizer and loss function
optimizer = optim.Adam(model.parameters(),
                       lr = hparams['learning_rate'],
                       weight_decay = hparams['weight_decay'])

loss_func = torch.nn.BCELoss()

# train
epochs = hparams['epochs']
gclip = hparams['gclip']
train_model(model, train_loader, val_loader, loss_func, optimizer, writer, gclip, epochs)

# save the loss on tensorboard
writer.close()


########################################################################
#                           END OF YOUR CODE                           #
########################################################################

cuda


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## Testing the Model

As you trained a model and improved it on the validation set, you can now test it on the test set.

In [ ]:
test_loader = DataLoader(
  test_dataset, batch_size=8, collate_fn=collate, drop_last=False
)

a, b, test_loss_history = compute_accuracy(model, test_loader, test_loss_history)
print("accuracy on test set: {}".format())


test_loss_history = [x.cpu() for x in test_loss_history]
test_loss_history[0]
# plt.plot(test_loss_history, label = "Test Loss")
# plt.xlabel("Iteration")
# plt.ylabel("Loss")
# plt.legend()
# plt.title('Training and Validation Loss')
# plt.show()


## Demo


Now that you trained a sufficiently good sentiment classifier, run the below cell and type some text to see some predictions (type exit to quit the demo). Since we used a small data, don't expect too much :).
<div class="alert alert-warning">
<h3>Warning!</h3>
    <p>
        As there is a while True loop in the cell below, you can skip this one for now and run the cell under '3. Submission' first to save your model.
   </p>
</div>

In [ ]:
plt.plot(val_loss_history, label = "Validation Loss")
plt.plot(train_loss_history, label = "Train Loss")
plt.plot(lr_history, label = "Learning Rate")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
from exercise_code.rnn.sentiment_dataset import tokenize

text = ''
w2i = vocab
while True:
    text = input()
    if text == 'exit':
        break

    words = torch.tensor([
        w2i.get(word, w2i['<unk>'])
        for word in tokenize(text)
    ]).long().to(device).view(-1, 1)  # T x B

    pred = model(words).item()
    sent = pred > 0.5

    print('Sentiment -> {}, Confidence -> {}'.format(
        ':)' if sent else ':(', pred if sent else 1 - pred
    ))
    print()

fuc# 3. Submission

If you got sufficient performance on the test data, you are ready to upload your model to the [server](https://i2dl.vc.in.tum.de/submission/) . As always, let's first save your final model.

In [ ]:
from exercise_code.util.save_model import save_model

save_model(model, 'rnn_classifier.p')

Congrats, you finished the last I2DL exercise! One last time this semester, let's prepare the submission:

In [ ]:
# Now zip the folder for upload
from exercise_code.util.submit import submit_exercise

submit_exercise('../output/exercise11')

# 4. Submission Instructions

Congratulations! You've just built your first image classifier! To complete the exercise, submit your final model to our submission portal - you probably know the procedure by now.

1. Go on [our submission page](https://i2dl.vc.in.tum.de/submission/), register for an account and login. We use your matriculation number and send an email with the login details to the mail account associated. When in doubt, login into tum-online and check your mails there. You will get an id which we need in the next step.
2. Log into [our submission page](https://i2dl.vc.in.tum.de/submission/) with your account details and upload the zip file.
3. Your submission will be evaluated by our system and you will get feedback about the performance of it. You will get an email with your score as well as a message if you have surpassed the threshold.
4. Within the working period, you can submit as many solutions as you want to get the best possible score.

# 5. Submission Goals

- Goal: Implement and train a recurrent neural network for sentiment analysis.
- Passing Criteria: Reach **Accuracy >= 83%** on __our__ test dataset. The submission system will show you your score after you submit.

- Submission start: __January 26, 2023, 13.00__
- Submission deadline: __February 1, 2023, 15.59__
- You can make **$\infty$** submissions until the deadline. Your __best submission__ will be considered for bonus.

# [Exercise Review](https://docs.google.com/forms/d/e/1FAIpQLSedSHEJ5vN-9FmJN-IGSQ9heDM_8qJQjHL4glgQGlrpQJEYPQ/viewform?usp=pp_url&entry.999074405=Exercise+11:+RNNs)

We are always interested in your opinion. Now that you have finished this exercise, we would like you to give us some feedback about the time required to finish the submission and/or work through the notebooks. Please take the short time to fill out our [review form](https://docs.google.com/forms/d/e/1FAIpQLSedSHEJ5vN-9FmJN-IGSQ9heDM_8qJQjHL4glgQGlrpQJEYPQ/viewform?usp=pp_url&entry.999074405=Exercise+11:+RNNs) for this exercise so that we can do better next time! :)